# Rating BCE loss


In [186]:
import copy
import os
import warnings
from ast import literal_eval
from typing import Any

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torcheval.metrics.functional.ranking import retrieval_precision
from tqdm import tqdm

In [2]:
MANUAL_SEED = 42
torch.manual_seed(MANUAL_SEED)

warnings.filterwarnings("ignore")

## Data loading and preprocessing


In [4]:
def load_dataset(path: str) -> pd.DataFrame:
    loaded_dfs = [
        pd.read_csv(os.path.join(path, file_name)) for file_name in os.listdir(path)
    ]
    return pd.concat(loaded_dfs)


def load_datasets(path: str) -> tuple[pd.DataFrame, pd.DataFrame]:
    return load_dataset(os.path.join(path, "train/")), load_dataset(
        os.path.join(path, "test/")
    )

In [5]:
train_df, val_df = load_datasets("../data/interim/masks_split/")

print(f"{len(train_df)=}")
print(f"{len(val_df)=}")

len(train_df)=22632
len(val_df)=2829


In [6]:
NUM_MOVIES = 1682
BASIC_USER_FEATURES = 3

TOTAL_USER_FEATURES = BASIC_USER_FEATURES + 19

In [19]:
class RecommendationDataset(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame):
        self.df = df.drop(columns=["user_id"])
        features = []
        inputs = []
        targets = []
        for _, row in tqdm(df.iterrows(), total=len(df)):
            features.append(row[:3].tolist() + literal_eval(row["genres"]))
            inputs.append(literal_eval(row["input"]))
            targets.append(literal_eval(row["output"]))

        self.features = np.array(features)

        # normalize ratings
        self.inputs = np.array(inputs) / 5
        self.targets = np.array(targets) / 5

    def __getitem__(self, idx: int) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
        input_ratings = self.inputs[idx]
        input_data = np.concatenate([self.features[idx], input_ratings])
        mask = input_ratings == 0
        return input_data, mask, self.targets[idx]

    def __len__(self) -> int:
        return len(self.df)

In [20]:
train_dataset, val_dataset = (
    RecommendationDataset(train_df),
    RecommendationDataset(val_df),
)
print(f"{len(train_dataset)=}")
print(f"{len(val_dataset)=}")

100%|██████████| 2829/2829 [00:27<00:00, 101.25it/s]


len(train_dataset)=22632
len(val_dataset)=2829


In [23]:
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")


DEVICE

device(type='cuda')

In [24]:
def collate_batch(batch: list) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    input_data_batch, mask_batch, target_batch = [], [], []
    for input_data, mask, target in batch:
        input_data_batch.append(input_data)
        mask_batch.append(mask)
        target_batch.append(target)

    return (
        torch.Tensor(input_data_batch),
        torch.Tensor(mask_batch).bool(),
        torch.Tensor(target_batch),
    )


train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
val_dataloader = torch.utils.data.DataLoader(
    dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch
)

In [25]:
it = train_dataloader._get_iterator()
inp, mask, out = it._next_data()
print(inp.shape)
print(mask.shape)
print(out.shape)

torch.Size([32, 1704])
torch.Size([32, 1682])
torch.Size([32, 1682])


## Creating the network


In [34]:
INPUT_SIZE = TOTAL_USER_FEATURES + NUM_MOVIES


class RecSys(nn.Module):
    def __init__(
        self,
        hidden_dim1: int = 1024,
        hidden_dim2: int = 1024,
    ):
        super(RecSys, self).__init__()

        self.d1 = nn.Dropout(0.1)

        self.fc1 = nn.Linear(INPUT_SIZE, hidden_dim1)

        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)

        self.fc3 = nn.Linear(hidden_dim2, NUM_MOVIES)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.d1(x)

        x = F.relu(self.fc2(x))

        return F.sigmoid(self.fc3(x))

In [35]:
torch.manual_seed(MANUAL_SEED)


def create_model() -> tuple[nn.Module, Any]:
    model = RecSys()

    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
    model = model.to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    return model, optimizer


loss_fn = torch.nn.BCELoss()

## Train model


In [36]:
def train_one_epoch(
    model: nn.Module, loader, optimizer, loss_fn, epoch, use_mask: bool = True
):
    model.train()
    train_loss = 0.0
    total = 0

    loop = tqdm(
        loader,
        total=len(loader),
        desc=f"Epoch {epoch}: train",
        leave=True,
    )
    for batch in loop:
        input_data, mask, target = batch
        input_data, target, mask = (
            input_data.to(DEVICE),
            target.to(DEVICE),
            mask.to(DEVICE),
        )

        # forward pass and loss calculation
        outputs = model(input_data)

        # zero the parameter gradients
        optimizer.zero_grad()

        positive_targets = (target > 0).float()
        if use_mask:
            loss = loss_fn(
                torch.masked_select(outputs, mask),
                torch.masked_select(positive_targets, mask),
            )
        else:
            loss = loss_fn(outputs, positive_targets)

        # backward pass
        loss.backward()
        total += target.shape[1]

        # optimizer run
        optimizer.step()

        train_loss += loss.item()
        loop.set_postfix({"loss": train_loss / total})


def val_one_epoch(model: nn.Module, loader, loss_fn, epoch, use_mask: bool = True):
    loop = tqdm(
        loader,
        total=len(loader),
        desc=f"Epoch {epoch}: val",
        leave=True,
    )
    val_loss = 0.0
    total = 0
    with torch.no_grad():
        model.eval()  # evaluation mode
        for batch in loop:
            input_data, mask, target = batch
            input_data, target, mask = (
                input_data.to(DEVICE),
                target.to(DEVICE),
                mask.to(DEVICE),
            )

            outputs = model(input_data)

            positive_targets = (target > 0).float()
            if use_mask:
                loss = loss_fn(
                    torch.masked_select(outputs, mask),
                    torch.masked_select(positive_targets, mask),
                )
            else:
                loss = loss_fn(outputs, positive_targets)

            val_loss += loss.item()
            total += target.shape[1]
            loop.set_postfix({"loss": val_loss / total})
    return val_loss / total

In [37]:
NUM_EPOCHS = 5


def train_model(
    model: nn.Module,
    optimizer,
    loss_fn,
    train_dataloader,
    val_dataloader,
    save_path: str,
    use_mask: bool = True,
) -> nn.Module:
    best_loss = 1e10

    for epoch in range(1, NUM_EPOCHS + 1):
        train_one_epoch(
            model, train_dataloader, optimizer, loss_fn, epoch, use_mask=use_mask
        )
        val_loss = val_one_epoch(model, val_dataloader, loss_fn, epoch, use_mask=use_mask)
        if val_loss <= best_loss:
            val_loss = best_loss
            torch.save(model, save_path)

    return copy.deepcopy(model)

In [38]:
model, optimizer = create_model()
model_mask, optimizer_mask = create_model()

In [39]:
best = train_model(
    model,
    optimizer,
    loss_fn,
    train_dataloader,
    val_dataloader,
    "../models/rating_bce",
    use_mask=False,
)

Epoch 5: val: 100%|██████████| 89/89 [00:04<00:00, 19.67it/s, loss=4.12e-5]


RecSys(
  (d1): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=1704, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1682, bias=True)
)

In [40]:
best_mask = train_model(
    model_mask,
    optimizer_mask,
    loss_fn,
    train_dataloader,
    val_dataloader,
    "../models/rating_bce_mask",
    use_mask=True,
)

Epoch 5: val: 100%|██████████| 89/89 [00:04<00:00, 19.62it/s, loss=5.09e-5]


RecSys(
  (d1): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=1704, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1682, bias=True)
)

## Test models


In [41]:
model = torch.load("../models/rating_bce")
model.eval()

RecSys(
  (d1): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=1704, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1682, bias=True)
)

In [42]:
model_mask = torch.load("../models/rating_bce_mask")
model_mask.eval()

RecSys(
  (d1): Dropout(p=0.1, inplace=False)
  (fc1): Linear(in_features=1704, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=1682, bias=True)
)

In [43]:
def get_single_output(
    model: nn.Module,
    input_data: np.ndarray,
):
    with torch.no_grad():
        model.eval()
        input_tensor = torch.Tensor([input_data]).to(DEVICE)
        model_out = model(input_tensor)

    return model_out[0].cpu().numpy()

In [ ]:
def load_genres(path: str) -> list[str]:
    return pd.read_csv(
        os.path.join(path, "u.genre"),
        sep="|",
        header=None,
        names=["name", "genre_idx"],
        encoding="ISO-8859-1",
    )["name"].tolist()


def load_items(path: str, genres: list[str]) -> pd.DataFrame:
    return pd.read_csv(
        os.path.join(path, "u.item"),
        sep="|",
        header=None,
        names=[
            "movie_id",
            "movie_title",
            "release_date",
            "video_release_date",
            "IMDb_URL",
            *genres,
        ],
        encoding="ISO-8859-1",
    )


genres = load_genres("../data/raw/ml-100k/")
movies_df = load_items("../data/raw/ml-100k/", genres)

In [135]:
a = np.array([1, 2, 3, 0, 0, 0, 0, 5, 0])
b = np.array([10, 20, 30, 40, 50, 60, 70, 80, 90])

In [136]:
b[np.nonzero(a > 0)[0]] = 0
b

array([ 0,  0,  0, 40, 50, 60, 70,  0, 90])

In [173]:
def get_unseen_on_input_data(
    input_rating: np.ndarray, movie_ratings: np.ndarray
) -> np.ndarray:
    unseen_ratings = movie_ratings.copy()
    seen_indices = np.nonzero(input_rating > 0)[0]
    unseen_ratings[seen_indices] = 0
    return unseen_ratings


def calculate_genre_ratios(
    movie_indices: np.ndarray, items_df: pd.DataFrame
) -> np.ndarray:
    genres_sum = (
        items_df[items_df["movie_id"].isin(movie_indices + 1)]
        .iloc[:, 5:]
        .sum(axis=0)
        .to_numpy()
    )
    return genres_sum / genres_sum.sum()


def get_recommendations(
    model: nn.Module,
    encoded_age: float,
    encoded_gender: int,
    encoded_occupation: int,
    movie_indices: list[int],
    movies_df: pd.DataFrame,
    predicted_threshold: float,
    num_recs: int = 5,
) -> np.ndarray:
    movie_indices_shifted = np.array(movie_indices) - 1  # starting from 0

    movies_ratings = np.zeros(NUM_MOVIES)
    movies_ratings[movie_indices_shifted] = 1.0  # rating = 5
    input_vector = np.array(
        [
            encoded_age,
            encoded_gender,
            encoded_occupation,
            *calculate_genre_ratios(np.array(movie_indices_shifted), movies_df),
            *movies_ratings,
        ]
    )

    predictions = get_single_output(model, input_vector)
    predictions[predictions < predicted_threshold] = 0.0
    unseen_predictions = get_unseen_on_input_data(movies_ratings, predictions)

    movie_ids = np.argsort(-unseen_predictions) + 1

    unknown_idx = 267  # actual idx (from 1)
    movie_ids = np.delete(movie_ids, np.where(movie_ids == unknown_idx))

    return movie_ids[:num_recs]


def get_movie_titles(
    recommended_movies: np.ndarray, movies_df: pd.DataFrame
) -> list[str]:
    return [
        movies_df[movies_df["movie_id"] == movie_id]["movie_title"].to_list()[0]
        for movie_id in recommended_movies
    ]


def show_recommendations(
    models_set: list[tuple[str, nn.Module]],
    movies_set: list[tuple[str, list[int]]],
    predicted_threshold: float = 0.0,
):
    for movies_name, movies in movies_set:
        print(movies_name)
        for model_name, model in models_set:
            recommended_movies = get_recommendations(
                model, 0.21, 1, 19, movies, movies_df, predicted_threshold
            )
            print(f"{model_name:10}: {get_movie_titles(recommended_movies, movies_df)}")
        print()

In [174]:
models_set = [
    ("No mask", model),
    ("Mask", model_mask),
]

movies_set = [
    (
        "SCI-FI",
        [50, 257, 204, 181],
    ),  # Star Wars, MIB, Back to The Future, Return of the Jedi
    ("CARTOONS", [1, 225, 465, 501]),  # Toy Story, 101 Dalmatians, Jungle Book, Dumbo
    ("STAR TRACK", [222, 228, 380, 449]),  # Star Tracks
    ("PULP FICTION", [56]),  # Pulp Fiction
]

show_recommendations(models_set, movies_set)

SCI-FI
No mask   : ['Star Trek VI: The Undiscovered Country (1991)', 'Evil Dead II (1987)', 'Godfather, The (1972)', 'Indiana Jones and the Last Crusade (1989)', 'Four Weddings and a Funeral (1994)']
Mask      : ['Breaking the Waves (1996)', 'Lawrence of Arabia (1962)', 'Alien (1979)', 'Terminator, The (1984)', 'Raiders of the Lost Ark (1981)']

CARTOONS
No mask   : ['Contact (1997)', 'Raiders of the Lost Ark (1981)', 'Star Wars (1977)', 'Beavis and Butt-head Do America (1996)', 'Pulp Fiction (1994)']
Mask      : ['Boogie Nights (1997)', 'Highlander (1986)', 'Star Wars (1977)', "Ulee's Gold (1997)", 'Die Hard (1988)']

STAR TRACK
No mask   : ['In the Name of the Father (1993)', 'Indiana Jones and the Last Crusade (1989)', 'Contact (1997)', 'Scream (1996)', 'Crumb (1994)']
Mask      : ['Star Trek VI: The Undiscovered Country (1991)', 'Aliens (1986)', 'Lawrence of Arabia (1962)', 'Under Siege (1992)', 'Magnificent Seven, The (1954)']

PULP FICTION
No mask   : ['In the Name of the Father 

In [176]:
show_recommendations(models_set, movies_set, predicted_threshold=0.8)  # rating >= 4

SCI-FI
No mask   : ['Last of the Mohicans, The (1992)', 'Four Weddings and a Funeral (1994)', 'Fugitive, The (1993)', 'Terminator 2: Judgment Day (1991)', 'True Lies (1994)']
Mask      : ['Star Trek VI: The Undiscovered Country (1991)', '2001: A Space Odyssey (1968)', 'Highlander (1986)', 'Die Hard (1988)', "Ulee's Gold (1997)"]

CARTOONS
No mask   : ['Perfect World, A (1993)', 'U Turn (1997)', 'Star Trek III: The Search for Spock (1984)', 'Crumb (1994)', 'Star Trek VI: The Undiscovered Country (1991)']
Mask      : ['Alien (1979)', 'Lawrence of Arabia (1962)', 'Highlander (1986)', '2001: A Space Odyssey (1968)', 'Boogie Nights (1997)']

STAR TRACK
No mask   : ['Alien (1979)', 'Terminator, The (1984)', 'Contact (1997)', 'Under Siege (1992)', 'U Turn (1997)']
Mask      : ['Raiders of the Lost Ark (1981)', 'Star Trek III: The Search for Spock (1984)', 'Indiana Jones and the Last Crusade (1989)', 'Return of the Jedi (1983)', 'Event Horizon (1997)']

PULP FICTION
No mask   : ['2001: A Space

## Metrics

In [183]:
def generate_test_data(
    model: nn.Module, dataset: RecommendationDataset
) -> list[tuple[np.ndarray, np.ndarray]]:
    test_data = []

    for input_data, _, target in tqdm(dataset):
        predicted = get_single_output(model, input_data)

        input_ratings = input_data[TOTAL_USER_FEATURES:]
        unseen_predicted = get_unseen_on_input_data(input_ratings, predicted)
        unseen_target = get_unseen_on_input_data(input_ratings, target)
        test_data.append((unseen_target, unseen_predicted))

    return test_data

In [181]:
def get_top_args(x: np.ndarray, n: int) -> np.ndarray:
    return np.argsort(-x)[:n]


def top_intersection(target: np.ndarray, predicted: np.ndarray, top_n: int = 20):
    return list(
        set(get_top_args(target, top_n)).intersection(get_top_args(predicted, top_n))
    )


def top_k_intersections(
    data: list[tuple[np.ndarray, np.ndarray]], k: int, threshold: float = 0.0
) -> list[int]:
    intersections = []
    for unseen_target, unseen_predicted in data:
        nonzero_targets = unseen_target[unseen_target > threshold]
        relevant_predicted = unseen_predicted[unseen_predicted > threshold]
        intersections.append(
            len(top_intersection(nonzero_targets, relevant_predicted, k))
        )

    return intersections


def retrieval_precisions_on_k(
    data: list[tuple[np.ndarray, np.ndarray]], k: int
) -> list[int]:
    retrieval_precisions = []
    for unseen_target, unseen_predicted in data:
        nonzero_targets = unseen_target > 0
        relevant_predicted = unseen_predicted

        retrieval_precisions.append(
            retrieval_precision(
                torch.Tensor(relevant_predicted), torch.Tensor(nonzero_targets), k
            )
        )

    return retrieval_precisions


def average_precision_on_k(target: np.ndarray, predicted: np.ndarray, k: int) -> float:
    relevant_predicted = predicted.copy()
    if len(relevant_predicted) > k:
        relevant_predicted = relevant_predicted[:k]

    score = 0.0
    hits = 0

    for idx, x in enumerate(relevant_predicted):
        if x in target and x not in relevant_predicted[:idx]:
            hits += 1
            score += hits / (idx + 1.0)

    return score / min(len(target), k)


def map_on_k(targets: list[np.ndarray], predictions: list[np.ndarray], k: int) -> float:
    return np.mean(
        [
            average_precision_on_k(target, predicted, k)
            for target, predicted in zip(targets, predictions)
        ]
    )


def generate_total_data_lists(
    data: list[tuple[np.ndarray, np.ndarray]]
) -> tuple[list[np.ndarray], list[np.ndarray]]:
    all_targets = []
    all_predictions = []
    for unseen_target, unseen_predicted in data:
        nonzero_targets = unseen_target > 0
        all_targets.append(
            np.argsort(nonzero_targets)[len(nonzero_targets) - sum(nonzero_targets) :]
        )
        all_predictions.append(np.argsort(-unseen_predicted))

    return all_targets, all_predictions

In [187]:
def show_metrics(data: list[tuple[np.ndarray, np.ndarray]], ks: list[int]):
    all_targets, all_predictions = generate_total_data_lists(data)
    for k in ks:
        print(f"K={k}")
        intersections = top_k_intersections(data, k)
        retrieval_precisions = retrieval_precisions_on_k(data, k)
        map_score = map_on_k(all_targets, all_predictions, k)

        print(f"Mean top intersections: {np.mean(intersections)}")
        print(f"Mean retrieval precision: {np.mean(retrieval_precisions)}")
        print(f"MAP: {map_score}")
        print()

In [191]:
ks = [5, 10, 20, 50]

test_data = generate_test_data(model, val_dataset)
test_data_mask = generate_test_data(model_mask, val_dataset)

100%|██████████| 2829/2829 [00:04<00:00, 668.52it/s]


In [193]:
show_metrics(test_data, ks)

K=5
Mean top intersections: 0.11947684694238246
Mean retrieval precision: 0.839802086353302
MAP: 0.8038234947566866

K=10
Mean top intersections: 0.3407564510427713
Mean retrieval precision: 0.7945916652679443
MAP: 0.7420682191811566

K=20
Mean top intersections: 1.0420643336868152
Mean retrieval precision: 0.7350654006004333
MAP: 0.6761280522947515

K=50
Mean top intersections: 4.363379285966773
Mean retrieval precision: 0.6175397634506226
MAP: 0.6387461661720604



In [192]:
show_metrics(test_data_mask, ks)

K=5
Mean top intersections: 0.11134676564156946
Mean retrieval precision: 0.7874161005020142
MAP: 0.7343831742665253

K=10
Mean top intersections: 0.3336868151290209
Mean retrieval precision: 0.7383174300193787
MAP: 0.669376525442273

K=20
Mean top intersections: 1.1081654294803818
Mean retrieval precision: 0.6741604804992676
MAP: 0.5986471207347243

K=50
Mean top intersections: 4.361258395192648
Mean retrieval precision: 0.5633792877197266
MAP: 0.552120947808151

